In [ ]:
import json
from requests import request
from sklearn import model_selection, preprocessing
import streamlit as st
import yfinance as yf
import streamlit.components.v1 as components
from django.shortcuts import render
import plotly.graph_objects as go
from plotly.offline import plot
import datetime as dt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import tickers
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import datetime
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split



try:
    df_ml = yf.download(tickers=ticker_value, period='3mo', interval='1h')
except: 
    ticker_value = 'AAPL'
    df_ml = yf.download(tickers=ticker_value, period='3mo', interval='1m')
    
    df_ml = df_ml[['Adj Close']]
    forecast_out = int(10)
    df_ml['Prediction'] = df_ml[['Adj Close']].shift(-forecast_out)
    # Splitting data for Test and Train
    X = np.array(df_ml.drop(['Prediction'],1))
    X = preprocessing.scale(X)
    X_forecast = X[-forecast_out:]
    X = X[:-forecast_out]
    y = np.array(df_ml['Prediction'])
    y = y[:-forecast_out]
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)
    # Applying Linear Regression
    clf = LinearRegression()
    clf.fit(X_train,y_train)
    # Prediction Score
    confidence = clf.score(X_test, y_test)
    # Predicting for 'n' days stock data
    forecast_prediction = clf.predict(X_forecast)
    forecast = forecast_prediction.tolist()
    # st.write(f'Ticker Symbol: {ticker_value}')
    # st.write(f'Number of Days to Forecast: {number_of_days}')
    # st.write(f'Confidence Score: {confidence}')
    # st.write(f'Forecast: {forecast}')

    pred_dict = {"Date": [], "Prediction": []}
    for i in range(0, len(forecast)):
        pred_dict["Date"].append(dt.datetime.today() + dt.timedelta(days=i))
        pred_dict["Prediction"].append(forecast[i])
    
    pred_df = pd.DataFrame(pred_dict)
    pred_fig = go.Figure([go.Scatter(x=pred_df['Date'], y=pred_df['Prediction'])])
    pred_fig.update_xaxes(rangeslider_visible=True)
    pred_fig.update_layout(paper_bgcolor="#14151b", plot_bgcolor="#14151b", font_color="white")
    plot_div_pred = plot(pred_fig, auto_open=False, output_type='div')
    
    st.plotly_chart(pred_fig)